# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил(а): Манахова Мария

### Общая информация
Дата выдачи: 

Дедлайн: 

### О задании

В этом домашнем задании вы будете работать с линейной классификацией, попрактикуетесь на реальной задаче классификации текстов.

Для решения этого домашнего задания намного удобнее будет использовать Colab, так как данных много.

### Оценивание и штрафы

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи
Загрузка файлов с решениями происходит в системе Anytask.

### Подготовка данных

In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [2]:
# Датасет можно скачать здесь

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv

--2020-04-26 17:59:03--  https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/tg55q9mrziroyrs/train_subset.csv [following]
--2020-04-26 17:59:03--  https://www.dropbox.com/s/raw/tg55q9mrziroyrs/train_subset.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce60a0c76ada0f516e975d85e66.dl.dropboxusercontent.com/cd/0/inline/A2m57Tl9Pc7viFtozwIKvVdjiS4azOncyTrFtndZre-6px-nc2eRsjf7elLsxzsyEjuI5XAGbkU2-NPhAQ7XYhRTd-2dzfC0xBjkB0ZEVovYaSk84Pn674ICce3pfSoN9Fs/file# [following]
--2020-04-26 17:59:03--  https://uce60a0c76ada0f516e975d85e66.dl.dropboxusercontent.com/cd/0/inline/A2m57Tl9Pc7viFtozwIKvVdjiS4azOncyTrFtndZre-6px-nc2eRsjf7elLsxzsyEjuI5XAGbkU2-NPhAQ7XYhRTd-2dzfC0xBj

### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title
 - description
 - Category_name
 - Category

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [3]:
data = pd.read_csv("train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [4]:
data.shape

(30000, 4)

In [0]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

### Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Давайте пока остановимся на простом WordPunctTokenizer.


In [8]:
from nltk.tokenize import WordPunctTokenizer


tokenizer = WordPunctTokenizer()


def preprocess(text: str) -> str:
    return ' '.join(tokenizer.tokenize(text.lower()))


text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'
print("before:", text,)
print("after:", preprocess(text),)

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...


__Задание:__ Токенизируйте title и description в train и test

In [0]:
def tokenize(sample):
  for i in range(sample.shape[0]):
    for j in range(sample.shape[1]):
      sample[i][j] = preprocess(sample[i][j])

tokenize(X_train)
tokenize(X_test)

In [10]:
X_train[:10]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ['пион ирис ромашка рассада',
        'пион куст 500 р ( более 10 шт )/ саженец / корень 100р / растут у нас более 70 лет / розовые , бордовые и белые / на фото цветы 2018г / п .

In [0]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

### BOW (1.5 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в train data
 - Для каждого примера из train посчитать, сколько раз каждое слово из словаря в нём встречается


 В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

__Задание:__ Найдите k самых частых слов, отсортируйте их по убыванию частотности (k=10000)

In [0]:
from collections import Counter

def get_vocabulary(sample, k):
  c = Counter()
  for i in range(sample.shape[0]):
    for j in range(sample.shape[1]):
      words = sample[i][j].split()
      for word in words:
        c[word] += 1

  return np.array(c.most_common(k))[:, 0]
  
bow_vocabulary = get_vocabulary(X_train, 10000)

In [0]:
assert sorted(bow_vocabulary)[::200] == ['!', '12500', '270', '700', 'by', 'gh', 'michael', 'sonata', 'ø', 'аудиоподготовка', 'большим', 'веса', 'воспроизведения', 'габариты', 'гтд', 'джинсами', 'доступность', 'загрузки', 'зимней', 'использовался', 'квартала', 'коммуникации', 'кошки', 'лакированные', 'магазин', 'металл', 'мск', 'натуральным', 'носке', 'одному', 'отвечаем', 'пассат', 'плотно', 'покраску', 'постоянные', 'примеры', 'просьба', 'размещайте', 'репетитор', 'сантехник', 'сидения', 'современного', 'стала', 'схема', 'тон', 'удлиненная', 'фасад', 'цветами', 'шея', 'эту']

In [0]:
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    """ 
    bow = {}
    for bow_word in bow_vocabulary:
      bow[bow_word] = 0
    words = text.split()
    for word in words:
      if word in bow:
        bow[word] += 1

    return np.fromiter(bow.values(), dtype=int)

In [0]:
assert np.allclose(np.where(text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml") != 0)[0],
                   np.array([   1,    4,   12,  565,  866, 1601, 2539, 4063])
)

In [0]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    # assert ниже написан для bow из description
    bow = []
    for item in items:
      bow.append(text_to_bow(item[1]))

    return np.array(bow)

In [0]:
assert np.allclose(np.where(items_to_bow([X_train[42]])[0] != 0),
                   np.array([   0, 1, 2, 5, 6, 7, 12, 27, 41, 49, 110,
                                189,  208,  221, 2032, 3052, 7179, 9568]),
)

In [0]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

In [0]:
from sklearn.metrics import accuracy_score

### Логистическая регрессия и SVC (1 балл)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.


In [0]:
from sklearn.linear_model import LogisticRegression
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow, y_train)
print(accuracy_score(bow_model.predict(X_test_bow), y_test))

assert accuracy_score(bow_model.predict(X_test_bow), y_test) > 0.7

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7011111111111111


In [0]:
from sklearn.svm import LinearSVC
bow_model = LinearSVC(max_iter=70).fit(X_train_bow, y_train)
print(accuracy_score(bow_model.predict(X_test_bow), y_test))

assert accuracy_score(bow_model.predict(X_test_bow), y_test) > 0.68

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.6848888888888889


Качество на логистической регрессии – 0.701, на линейной классификации методом опорных векторов – 0.685.

### Модификация признаков (0.5 балла)

Добавьте title товара в bow с произвольным весом, как изменится качество?

In [0]:
def items_to_bow_title(items: np.array) -> np.array:
    bow = []
    for item in items:
      bow.append(text_to_bow(item[0]))
    return np.array(bow)

X_train_bow_with_title = X_train_bow + items_to_bow_title(X_train) * 2
X_test_bow_with_title = X_test_bow + items_to_bow_title(X_test) * 2

In [0]:
bow_model_with_title = LogisticRegression(max_iter=100).fit(X_train_bow_with_title, y_train)
print(accuracy_score(bow_model_with_title.predict(X_test_bow_with_title), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7944444444444444


In [0]:
bow_model_with_title = LinearSVC(max_iter=70).fit(X_train_bow_with_title, y_train)
print(accuracy_score(bow_model_with_title.predict(X_test_bow_with_title), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7621111111111111


После добавления title в bow с весом 2 качество модели улучшилось и на логистической регрессии – 0.794 (на 0.093), и на линейной классификации методом опорных векторов – 0.762 (на 0.077). 

Нормализуйте данные (`sklearn.preprocessing.normalize`) перед обучением. Что станет с качеством и почему?

In [0]:
from sklearn import preprocessing
X_train_bow_normalized = preprocessing.normalize(X_train_bow_with_title)
X_test_bow_normalized = preprocessing.normalize(X_test_bow_with_title)

In [0]:
bow_model_normalized = LogisticRegression(max_iter=100).fit(X_train_bow_normalized, y_train)
print(accuracy_score(bow_model_normalized.predict(X_test_bow_normalized), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7041111111111111


In [0]:
bow_model_normalized = LinearSVC(max_iter=70).fit(X_train_bow_normalized, y_train)
print(accuracy_score(bow_model_normalized.predict(X_test_bow_normalized), y_test))

0.8072222222222222


После нормализации данных качество на линейной регрессии упало по сравнению с не нормализованными данными – 0.704 (на 0.09), качество обучения с использованием линейной классификации методом опорных векторов возросло – 0.807 (на 0.045). Тем не менее, по сравнению с исходными данными (без учета title) качество на обеих моделях все еще лучше.

### mystem (0.5) балла

Попробуйте обучиться, используя токенизатор mystem. Сравните качество.

In [0]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

--2020-04-26 17:11:10--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.243, 5.45.205.242, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm902.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-04-26 17:11:10--  http://cache-mskm902.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm902.cdn.yandex.net (cache-mskm902.cdn.yandex.net)... 5.45.220.12, 2a02:6b8:0:2002::13
Connecting to cache-mskm902.cdn.yandex.net (cache-mskm902.cdn.yandex.net)|5.45.220.12|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  1

In [0]:
pip install git+https://github.com/nlpub/pymystem3

  Cloning https://github.com/nlpub/pymystem3 to /tmp/pip-req-build-zmmdo5v_
  Running command git clone -q https://github.com/nlpub/pymystem3 /tmp/pip-req-build-zmmdo5v_
  Created wheel for pymystem3: filename=pymystem3-0.2.0-cp36-none-any.whl size=9921 sha256=8d4425f01795e916d2436a76c99863fb5bb673d5b64eded74d4e82795b66d987
  Stored in directory: /tmp/pip-ephem-wheel-cache-blctvvj1/wheels/7d/75/c2/216a594291dee680749ce12c60d16125cfe1f363059e7163dc
Successfully built pymystem3


In [0]:
from pymystem3 import Mystem

In [0]:
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X, y, test_size=0.3, random_state=42)

In [0]:
mystem = Mystem()

def mystem_preprocess(text: str) -> str:
    return ' '.join(mystem.lemmatize(text.lower()))

In [0]:
def lemmatize(sample):
  for i in range(sample.shape[0]):
    for j in range(sample.shape[1]):
      sample[i][j] = mystem_preprocess(sample[i][j])

In [0]:
lemmatize(X_train_m)
lemmatize(X_test_m)

In [0]:
bow_vocabulary_m = get_vocabulary(X_train_m, 10000)

In [0]:
def text_to_bow_m(text: str, bow_vocabulary) -> np.array:
    bow = {}
    for bow_word in bow_vocabulary:
      bow[bow_word] = 0
    words = text.split()
    for word in words:
      if word in bow:
        bow[word] += 1

    return np.fromiter(bow.values(), dtype=int)

In [0]:
def items_to_bow_m(items: np.array, bow_vocabulary) -> np.array:
    bow = []
    for item in items:
      bow.append(text_to_bow_m(item[1], bow_vocabulary))

    return np.array(bow)

In [0]:
X_train_m_bow = items_to_bow_m(X_train_m, bow_vocabulary_m)
X_test_m_bow = items_to_bow_m(X_test_m, bow_vocabulary_m)

In [0]:
bow_model_m = LogisticRegression(max_iter=100).fit(X_train_m_bow, y_train_m)
print(accuracy_score(bow_model_m.predict(X_test_m_bow), y_test_m))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7298888888888889


In [0]:
bow_model_m = LinearSVC(max_iter=70).fit(X_train_m_bow, y_train_m)
print(accuracy_score(bow_model_m.predict(X_test_m_bow), y_test_m))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7072222222222222


Качество на логистической регресси – 0.73, на линейной классификации методом опорных векторов – 0.707.

При использовании токенизатора mystem качество по сравнению с использованием токенизатора WordPunctTokenizer возросло и на логистической регрессии (на 0.029), и на линейной классификации методом опорных векторов (на 0.022).

### TF-IDF (1.5 балла)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя.

In [0]:
# Давайте для простоты считать один tf-idf для title и description.
# Для каждого слова из bow_vocabulary нужно посчитать
# в тексте скольких товаров встретилось это слово

def count(sample, vocabulary):
  vocabulary_count = {}
  for word in vocabulary:
    vocabulary_count[word] = 0

  for i in range(sample.shape[0]):
      words = set(' '.join([sample[i][0], sample[i][1]]).split())
      for word in words:
        if word in vocabulary_count:
          vocabulary_count[word] += 1

  return vocabulary_count

count_dict = count(X_train, bow_vocabulary)

In [0]:
def get_idf(count_dict, total_number_of_documents) -> np.array:
  idf_array = []
  for term in count_dict:
    idf_array.append(np.log(total_number_of_documents / count_dict[term]))

  return np.array(idf_array)

idf = get_idf(count_dict, X_train.shape[0])

In [0]:
def get_tf(text, vocabulary):
  words = text.split()
  frequences = Counter()
  for word in words:
    frequences[word] += 1

  tf_array = []
  for term in vocabulary:
    if term in words:
      tf_array.append(frequences[term] / len(set(words)))
    else:
      tf_array.append(0)

  return np.array(tf_array)

In [0]:
def text_to_tfidf(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указан tf-idf
    """
    tf = get_tf(text, bow_vocabulary)
    tfidf = []

    for i in range(idf.shape[0]):
      tfidf.append(idf * tf[i])

    return np.array(tfidf)

In [0]:
def items_to_tfidf(items: np.array) -> np.array:
    tfidf = []
    for item in items:
      tfidf.append(text_to_tfidf(' '.join([item[0], item[1]])))

    return np.array(tfidf)

In [0]:
X_train_tfidf = items_to_tfidf(X_train)
X_test_tfidf = items_to_tfidf(X_test)

In [0]:
# Нормализуйте данные
X_train_tfidf = preprocessing.normalize(X_train_tfidf)
X_test_tfidf = preprocessing.normalize(X_test_tfidf)

### Модели на TF-IDF признаках (1 балл)

Обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

In [0]:
from sklearn.linear_model import LogisticRegression
bow_model = LogisticRegression(max_iter=100).fit(X_train_tfidf, y_train)
print(accuracy_score(bow_model.predict(X_test_tfidf), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7653333333333333


In [0]:
bow_model = LinearSVC(max_iter=70).fit(X_train_tfidf, y_train)
print(accuracy_score(bow_model.predict(X_test_tfidf), y_test))

0.8067777777777778


По сравнению с исходным качество возросло и на логистической регрессии – 0.765 (на 0.064), и на линейной классификации методом опорных векторов – 0.807 (на 0.12).

### Hashing Vectorizer (0.5 балла)

Попробуйте использовать `sklearn.feature_extraction.text.HashingVectorizer` для векторизации текстов.
Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [0]:
from sklearn.feature_extraction.text import HashingVectorizer

In [0]:
def list_from_data(data):
  data_list = []
  for i in range(data.shape[0]):
    data_list.append(' '.join([data[i][0], data[i][1]]))

  return data_list

In [0]:
X_train_hv, X_test_hv, y_train_hv, y_test_hv = train_test_split(X, y, test_size=0.3, random_state=42)

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
vectorizer = HashingVectorizer()
X_train_hv = vectorizer.fit_transform(list_from_data(X_train_hv))
X_test_hv = vectorizer.fit_transform(list_from_data(X_test_hv))

In [18]:
hv_model = LinearSVC(max_iter=70).fit(X_train_hv, y_train_hv)
print(accuracy_score(hv_model.predict(X_test_hv), y_test_hv))

0.8245555555555556


In [0]:
hv_model = LogisticRegression(max_iter=100).fit(X_train_hv, y_train_hv)
print(accuracy_score(hv_model.predict(X_test_hv), y_test_hv))

Качество на линейной классификации методом опорных векторов возросло по сравнению со всеми предыдущими алгоритмами и их модификациями – 0.825, качество на логистической регрессии также показало неплохой результат.

### Word Vectors (1 балл)

Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах их интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [23]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

--2020-04-26 18:10:51--  https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0x7oxso6x93efzj/ru.tar.gz [following]
--2020-04-26 18:10:51--  https://www.dropbox.com/s/raw/0x7oxso6x93efzj/ru.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc5c546df22557a9302470bd8af.dl.dropboxusercontent.com/cd/0/inline/A2me129Rph9v5Ay3z2GQMjy9fCfT6SqX63eANWHnUtLg5yx_ifSjDqdHnVmQ93R9jCCU2CrtAvi_MFclDRfp8Tr3GNzU2k8U8xz3Ou1MjRU9zw/file# [following]
--2020-04-26 18:10:52--  https://ucc5c546df22557a9302470bd8af.dl.dropboxusercontent.com/cd/0/inline/A2me129Rph9v5Ay3z2GQMjy9fCfT6SqX63eANWHnUtLg5yx_ifSjDqdHnVmQ93R9jCCU2CrtAvi_MFclDRfp8Tr3GNzU2k8U8xz3Ou1MjRU9zw/file
Resolving ucc5c546df22557

In [24]:
!tar -xzvf ru.tar.gz

ru.bin
ru.vec


In [0]:
import gensim
from gensim.models.wrappers import FastText


model = FastText.load_fasttext_format('ru.bin')

In [0]:
# Эмбеддинг предложения -- сумма эмбеддингов токенов

def sentence_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """

    embedding_dim = model['кек'].shape[0]
    features = np.zeros([embedding_dim], dtype='float32')
    
    for word in sentence.split():
        if word in model:
            features += model[word]
    
    return features

In [0]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

In [0]:
# Обучите логистическую регрессию и SVM
# Оцените качество (accuracy_score)
X_train_e, X_test_e, y_train_e, y_test_e = train_test_split(X, y, test_size=0.3, random_state=42)

In [0]:
def items_to_sentence_embedding(items: np.array):
  sentence_embeddings = []
  for item in items:
    sentence_embeddings.append(sentence_embedding(' '.join([item[0], item[1]])))

  return sentence_embeddings

In [0]:
X_train_e = items_to_sentence_embedding(X_train_e)
X_test_e = items_to_sentence_embedding(X_test_e)

In [31]:
model_e = LinearSVC(max_iter=70).fit(X_train_e, y_train_e)
print(accuracy_score(model_e.predict(X_test_e), y_test_e))

0.4816666666666667


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [32]:
model_e = LogisticRegression(max_iter=100).fit(X_train_e, y_train_e)
print(accuracy_score(model_e.predict(X_test_e), y_test_e))

0.5008888888888889


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Качество на обеих моделях сильно хуже, чем при использовании других подходов, логистическая регрессия – 0.5, линейная классификаци методом опорных векторов – 0.481).

### Что дальше?

Решение каждого пункта 1 балл:

1. N-Gram модели текстовой классификации

2. Обучиться на полных данных (контест на kaggle)

3. Поработать с другими эмбеддингами (word2vec, GloVe).

4. Использовать Vowpal Wabbit вместо sklearn.

5. Другие способы токенизации (pymorphy2, spaCy)


Снабжайте код пояснениями и графиками.
Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.

In [34]:
pip install pymorphy2

     |████████████████████████████████| 51kB 3.9MB/s 
     |████████████████████████████████| 7.1MB 28.6MB/s 


In [35]:
pip install -U pymorphy2-dicts-ru

     |████████████████████████████████| 8.0MB 9.7MB/s 


In [0]:
import pymorphy2
from pymorphy2.tokenizers import simple_word_tokenize
morph = pymorphy2.MorphAnalyzer()

In [0]:
def pymorphy_preprocess(text: str) -> str:
    return ' '.join(simple_word_tokenize(text.lower()))

In [0]:
def tokenize(sample):
  for i in range(sample.shape[0]):
    for j in range(sample.shape[1]):
      sample[i][j] = pymorphy_preprocess(sample[i][j])

In [0]:
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X, y, test_size=0.3, random_state=42)

In [0]:
tokenize(X_train_p)
tokenize(X_test_p)

In [0]:
bow_vocabulary_p = get_vocabulary(X_train_p, 10000)

In [0]:
X_train_p_bow = items_to_bow_m(X_train_p, bow_vocabulary_p)
X_test_p_bow = items_to_bow_m(X_test_p, bow_vocabulary_p)

In [71]:
model_p = LinearSVC(max_iter=70).fit(X_train_p_bow, y_train_p)
print(accuracy_score(model_p.predict(X_test_p_bow), y_test_p))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.598


In [72]:
model_e = LogisticRegression(max_iter=100).fit(X_train_e, y_train_e)
print(accuracy_score(model_e.predict(X_test_e), y_test_e))

0.5008888888888889


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
